In [57]:
from Parallelizer import make_parallel
from common import  Common
import re
import pandas as pd
import time
from urllib.parse import unquote
from utils import *
import json
from math import ceil


# TIKTOK

In [58]:
import requests
from bs4 import BeautifulSoup
from time import sleep
count = 0

@make_parallel
def request_tiktok(url):
    payload = {}
    headers = {
        'authority': 'oec-api.tiktokv.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en;q=0.9,en-US;q=0.8',
        'cache-control': 'max-age=0',
        'sec-ch-ua': '"Microsoft Edge";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.47'
    }
    try_time = 0
    json_obj = None
    while not json_obj:
        
        # url = f"https://oec-api.tiktokv.com/view/product/{product_id}"
        response = requests.request("GET", url, headers=headers, data=payload)
        try_time += 1
        print(f"{try_time}: {url}")
        try:
            soup = BeautifulSoup(response.text, 'html.parser')

            # get content of script id = RENDER_DATA
            script = soup.find('script', id='RENDER_DATA')
            # get content of script
            script_content = script.contents[0]
            # get data from script
            # print(response.text)
            json_obj = json.loads(unquote(script_content))
            # print(json_obj)
        except Exception as e:
            print(e)
            json_obj = None
            print(f"fail: {url} {response.text}")
            sleep(2)
    return json_obj

In [59]:
@make_parallel
def transform(response, results=[]):

    product_transformed = {
        "product_id": "failed",
        "name": "failed",
        "link": "failed",
        "type": "Sp lẻ",
        "min_price": "failed",
        "max_price": "failed",
        "sold_count": "failed",
        "min_original_price": "failed",
        "max_original_price": "failed",
    }

    for i in range(5):
        product_transformed[f"image_{i+1}"] = "failed"

    try:
        product_base = response['2']['initialData']['productInfo']['product_base']
        mapping_product_base = {
            "real_price": ["price", "real_price"],
            "original_price": ["price", "original_price"],
            "images": ["images"],
            "sold_count": ["sold_count"],
            "title": ["title"]
        }
        transformed_product_base = Common.mapping_data(
            product_base, mapping_product_base)
        product_id = response['2']['initialData']['productInfo']['product_id']
        print(product_id)
        name = transformed_product_base['title']
        price_pattern = re.compile(r'(\d+\.?\d*)')
        try:
            is_flash_sale = bool(
                response['2']['initialData']['productInfo'].get("flash_sale", False)
            )
            prices = price_pattern.findall(transformed_product_base['real_price'].replace('.', ''))
            if len(prices) == 2:
                min_price = int(prices[0].replace('.', ''))
                max_price = int(prices[1].replace('.', ''))
            else:
                min_price = None
                max_price = int(prices[0].replace('.', ''))
            original_price = price_pattern.findall(
                transformed_product_base['original_price'])
            if len(original_price) == 2:
                min_original_price = int(original_price[0].replace('.', ''))
                max_original_price = int(original_price[1].replace('.', ''))
            else:
                min_original_price = None
                if len(original_price) == 0:
                    max_original_price = None
                else:
                    max_original_price = int(original_price[0].replace('.', ''))
        except Exception as e:
            print("not found price")
            min_price = None
            max_price = None
            min_original_price = None
            max_original_price = None

        sold_count = transformed_product_base['sold_count']

        images = [None]*5
        for i,image in enumerate(transformed_product_base["images"][:5]):
            images[i] = f'=image("{image["thumb_url_list"][0]}")'

        product_transformed = {
            "product_id": product_id,
            "name": name,
            "link": f"https://oec-api.tiktokv.com/view/product/{product_id}",
            "type": "Sp lẻ",
            "min_price": min_price,
            "max_price": max_price,
            "sold_count": sold_count,
            "min_original_price": min_original_price,
            "max_original_price": max_original_price,
        }
        
        if is_flash_sale:
            if min_original_price is not None:
                product_transformed["min_price"] = min_original_price
            if max_original_price is not None:
                product_transformed["max_price"] = max_original_price

        for i in range(5):
            product_transformed[f"image_{i+1}"] = images[i]

    except Exception as e:
        print(f"fail: {e}")
        if product_id:=response['_location'].split('/')[-1]:
            product_transformed["product_id"] = product_id
            product_transformed['link'] =  f"https://oec-api.tiktokv.com/view/product/{product_id}"
            print(f"failed {product_id}")

    results.append(product_transformed)

In [60]:
links =['https://oec-api.tiktokv.com/view/product/1729443992011574237',
'https://oec-api.tiktokv.com/view/product/1729444867137375197',
'https://oec-api.tiktokv.com/view/product/1729445344721603549',
'https://oec-api.tiktokv.com/view/product/1729445996211178461',
'https://oec-api.tiktokv.com/view/product/1729446330084068317',
'https://oec-api.tiktokv.com/view/product/1729446700188535773',
'https://oec-api.tiktokv.com/view/product/1729446728967359453',
'https://oec-api.tiktokv.com/view/product/1729446754308557789',
'https://oec-api.tiktokv.com/view/product/1729447195035929565',
'https://oec-api.tiktokv.com/view/product/1729447205246766045',
'https://oec-api.tiktokv.com/view/product/1729447485106326493',
'https://oec-api.tiktokv.com/view/product/1729447489862011869',
'https://oec-api.tiktokv.com/view/product/1729447522836974557',
'https://oec-api.tiktokv.com/view/product/1729447609082743773',
'https://oec-api.tiktokv.com/view/product/1729447612949433309',
'https://oec-api.tiktokv.com/view/product/1729447615382588381',
'https://oec-api.tiktokv.com/view/product/1729447618324564957',
'https://oec-api.tiktokv.com/view/product/1729447619223522269',
'https://oec-api.tiktokv.com/view/product/1729447619682864093',
'https://oec-api.tiktokv.com/view/product/1729447621629545437',
'https://oec-api.tiktokv.com/view/product/1729447628823759837',
'https://oec-api.tiktokv.com/view/product/1729606944439897053',
'https://oec-api.tiktokv.com/view/product/1729610111159733213',
'https://oec-api.tiktokv.com/view/product/1729612334436355037',
'https://oec-api.tiktokv.com/view/product/1729633502355557341',
'https://oec-api.tiktokv.com/view/product/1729640641502743517',
'https://oec-api.tiktokv.com/view/product/1729651019080371165',
'https://oec-api.tiktokv.com/view/product/1729651063887465437',
'https://oec-api.tiktokv.com/view/product/1729651081008024541',
'https://oec-api.tiktokv.com/view/product/1729651097827249117',
'https://oec-api.tiktokv.com/view/product/1729651204056320989',
'https://oec-api.tiktokv.com/view/product/1729651212604312541',
'https://oec-api.tiktokv.com/view/product/1729651238576098269',
'https://oec-api.tiktokv.com/view/product/1729651650935819229',
'https://oec-api.tiktokv.com/view/product/1729651656980859869',
'https://oec-api.tiktokv.com/view/product/1729651676151516125',
'https://oec-api.tiktokv.com/view/product/1729651851473357789',
'https://oec-api.tiktokv.com/view/product/1729651862815083485',
'https://oec-api.tiktokv.com/view/product/1729651863008414685',
'https://oec-api.tiktokv.com/view/product/1729652334448905181',
'https://oec-api.tiktokv.com/view/product/1729652352965512157',
'https://oec-api.tiktokv.com/view/product/1729652358494587869',
'https://oec-api.tiktokv.com/view/product/1729652366910786525',
'https://oec-api.tiktokv.com/view/product/1729652370994990045',
'https://oec-api.tiktokv.com/view/product/1729652384317606877',
'https://oec-api.tiktokv.com/view/product/1729654954614295517',
'https://oec-api.tiktokv.com/view/product/1729654958393428957',
'https://oec-api.tiktokv.com/view/product/1729655090747247581',
'https://oec-api.tiktokv.com/view/product/1729655688804600797',
'https://oec-api.tiktokv.com/view/product/1729655692674763741',
'https://oec-api.tiktokv.com/view/product/1729655735442770909',
'https://oec-api.tiktokv.com/view/product/1729655738373802973',
'https://oec-api.tiktokv.com/view/product/1729655751811238877',
'https://oec-api.tiktokv.com/view/product/1729656551694830557',
'https://oec-api.tiktokv.com/view/product/1729656557905611741',
'https://oec-api.tiktokv.com/view/product/1729662230486289373',
'https://oec-api.tiktokv.com/view/product/1729662256450014173',
'https://oec-api.tiktokv.com/view/product/1729664711043156957',
'https://oec-api.tiktokv.com/view/product/1729664718308936669',
'https://oec-api.tiktokv.com/view/product/1729664739616459741',
'https://oec-api.tiktokv.com/view/product/1729664744538541021',
'https://oec-api.tiktokv.com/view/product/1729669566290365405',
'https://oec-api.tiktokv.com/view/product/1729670064714451933',
'https://oec-api.tiktokv.com/view/product/1729679819777346525',
'https://oec-api.tiktokv.com/view/product/1729679885311642589',
'https://oec-api.tiktokv.com/view/product/1729679946110372829',
'https://oec-api.tiktokv.com/view/product/1729679951332084701',
'https://oec-api.tiktokv.com/view/product/1729680070248270813',
'https://oec-api.tiktokv.com/view/product/1729680071138577373',
'https://oec-api.tiktokv.com/view/product/1729680862038232029',
'https://oec-api.tiktokv.com/view/product/1729680876379081693',
'https://oec-api.tiktokv.com/view/product/1729680900283206621',
'https://oec-api.tiktokv.com/view/product/1729680912532999133',
'https://oec-api.tiktokv.com/view/product/1729681849227578333',
'https://oec-api.tiktokv.com/view/product/1729681869064276957',
'https://oec-api.tiktokv.com/view/product/1729681905640573917',
'https://oec-api.tiktokv.com/view/product/1729681941929757661',
'https://oec-api.tiktokv.com/view/product/1729682752813763549',
'https://oec-api.tiktokv.com/view/product/1729683574522612701',
'https://oec-api.tiktokv.com/view/product/1729683725759122397',
'https://oec-api.tiktokv.com/view/product/1729683731610962909',
'https://oec-api.tiktokv.com/view/product/1729683745918520285',
'https://oec-api.tiktokv.com/view/product/1729683771287243741',
'https://oec-api.tiktokv.com/view/product/1729694164930759645',
'https://oec-api.tiktokv.com/view/product/1729694755435416541',
'https://oec-api.tiktokv.com/view/product/1729694855954992093',]

In [61]:
product_infos = request_tiktok(links)

1: https://oec-api.tiktokv.com/view/product/1729444867137375197
1: https://oec-api.tiktokv.com/view/product/1729445996211178461
1: https://oec-api.tiktokv.com/view/product/1729447615382588381
1: https://oec-api.tiktokv.com/view/product/17294453447216035491: https://oec-api.tiktokv.com/view/product/1729446330084068317

1: https://oec-api.tiktokv.com/view/product/1729443992011574237
1: https://oec-api.tiktokv.com/view/product/1729447489862011869
1: https://oec-api.tiktokv.com/view/product/1729446700188535773
1: https://oec-api.tiktokv.com/view/product/1729446728967359453
1: https://oec-api.tiktokv.com/view/product/1729446754308557789
1: https://oec-api.tiktokv.com/view/product/1729447522836974557
1: https://oec-api.tiktokv.com/view/product/1729447205246766045
1: https://oec-api.tiktokv.com/view/product/1729447609082743773
1: https://oec-api.tiktokv.com/view/product/1729447195035929565
1: https://oec-api.tiktokv.com/view/product/1729447612949433309
1: https://oec-api.tiktokv.com/view/prod

In [62]:
len(product_infos), len(links)

(86, 86)

In [64]:
results = []
transform(product_infos, results)

1729443992011574237
1729444867137375197
1729445344721603549
17294459962111784611729446330084068317
1729446700188535773

1729446728967359453
1729446754308557789
1729447195035929565
1729447205246766045
1729447485106326493
1729447489862011869
1729447522836974557
1729447609082743773
17294476129494333091729447615382588381
1729447618324564957
1729447619223522269
1729447619682864093
1729447621629545437

17294476288237598371729606944439897053
1729610111159733213
1729612334436355037
1729633502355557341
1729640641502743517
1729651019080371165

1729651063887465437
1729651081008024541
1729651097827249117
1729651204056320989
1729651212604312541
1729651238576098269
1729651650935819229
17296516569808598691729651676151516125
1729651851473357789
1729651862815083485

1729651863008414685
1729652334448905181
1729652352965512157
1729652358494587869
1729652366910786525
1729652370994990045
1729652384317606877
1729654954614295517
1729654958393428957
1729655090747247581
1729655688804600797
1729655692674763741


[]

In [65]:
brand = "sweetladymart"

In [66]:
df = pd.DataFrame(results)
df['type'] = 'Sp lẻ'
colums = ['product_id','name','link','type','min_price','max_price','sold_count','image_1','image_2','image_3','image_4','image_5','min_original_price','max_original_price','product_code']
df.shape

(86, 14)

In [67]:
import re

PATTERN = r'\b\d{3,}\b'
PATTERN = r"[A-Z]\d+|\sQ\d+|\sC\d+|\sQ\d+|\s\d{3,}"
PATTERN = r"\b[\w+]+\d+"

df['name'] = df['name'].apply(lambda x: x.replace('_', ' '))
df['product_code'] = df['name'].apply(lambda x: re.findall(PATTERN, x)[0] if len(re.findall(PATTERN, x)) > 0 else None)
df['product_code'] = df['product_code'].apply(lambda x: x.upper() if x else None)
# df['product_code'] = df['name'].apply(lambda x: x.split(' ')[-1])


In [68]:
df.to_csv(f"{brand}/tiktok.csv", index=False, columns=colums)


In [69]:
# count product_code not null
df[df['product_code'].notnull()].shape

(29, 15)

# Shopee

In [70]:
from pathlib import Path

In [71]:
all_items = []

In [72]:
with open(f"{brand}/all.json", "r", encoding="utf8") as f:
    all_cralwed = json.load(f)
    for crawled in all_cralwed:
        if data:=crawled.get('data'):
            all_items += crawled['data']['items']
        else:
            all_items += crawled['items']

In [73]:
results = []

In [74]:


def transform_product(item):
    try:
        if "item_basic" in item:
            item = item['item_basic']
        product_id = item['itemid']
        shop_id = item['shopid']
        name = item['name']
        link = transform_link(name,shop_id,product_id)
        max_price = item['price_max'] /100000
        min_price = item['price_min'] / 100000 if item['price_min'] and item['price_min'] !=item['price_max'] else None
        sold_count = int(round(item['historical_sold']/100) * 100) if item['historical_sold'] >=1000 else item['historical_sold']
        max_original_price = item['price_max_before_discount'] / 100000 if item['price_max_before_discount'] > 0 else None
        min_original_price = item['price_min_before_discount'] / 100000 if item['price_min_before_discount'] >0 and item['price_min_before_discount'] != item['price_max_before_discount']  else None
        images = [None]*5
        for i,image in enumerate(item["images"][:5]):
            images[i] = f'=image("https://down-vn.img.susercontent.com/file/{image}")'
            
        product_transformed = {
            "product_id": product_id,
            "name": name,
            "link": link,
            "type": "Sp lẻ",
            "min_price": min_price,
            "max_price": max_price,
            "sold_count": sold_count,
            "min_original_price": min_original_price,
            "max_original_price": max_original_price,
        }
        
        for i in range(5):
            product_transformed[f"image_{i+1}"] = images[i]
        return product_transformed
    except Exception as e:
        print(e)
        print(json.dumps(item, indent=4))
    

In [75]:
len(all_items)

347

In [76]:
transformed_products = []
for item in all_items:
    transformed_products.append(transform_product(item))

In [77]:
len(transformed_products)

347

In [78]:
# PATTERN = r"\b[A-Z]\d+|\sQ\d+|\sC\d+|\sQ\d+|\s\d{3,}\b"


In [79]:
df = pd.DataFrame(transformed_products)
columns = ["product_id","name","link","type","min_price","max_price","sold_count","image_1","image_2","image_3","image_4","image_5","min_original_price","max_original_price","product_code"]

df['type'] = 'Sp lẻ'
df['name'] = df['name'].apply(lambda x: x.replace('_', ' '))
df['product_code'] = df['name'].apply(lambda x: re.findall(PATTERN, x)[-1] if len(re.findall(PATTERN, x)) > 0 else None)
df['product_code'] = df['product_code'].apply(lambda x: x.upper() if x else None)
print(df.shape)
df.to_csv(f"{brand}/shopee.csv", index=False, columns=columns)

(347, 15)


In [80]:
# df['product_code'] = df['name'].apply(lambda x: x.split(' ')[-1])
df['product_code'] = df['name'].apply(lambda x: re.findall(PATTERN, x)[0] if len(re.findall(PATTERN, x)) > 0 else None)

In [81]:
df.shape

(347, 15)